# Users Domain Namespace Testing

This notebook tests the new Users Domain Namespace API implementation.
It demonstrates the cleaner API surface compared to the legacy methods.

In [ ]:
# Setup and imports
import os
import sys

sys.path.insert(0, os.path.join(os.getcwd(), "../src"))

from kili.client import Kili

In [ ]:
# Initialize Kili client with test credentials
API_KEY = ""
ENDPOINT = "http://localhost:4001/api/label/v2/graphql"

kili = Kili(
    api_key=API_KEY,
    api_endpoint=ENDPOINT,
    legacy=False,  # Use the new domain API
)

print("Kili client initialized successfully!")
print(f"Users namespace available: {hasattr(kili, 'users')}")

Kili client initialized successfully!
Users namespace available: True


## Test Users Domain Namespace Access

In [ ]:
# Access the users namespace
users = kili.users
print(f"Users namespace type: {type(users)}")
print(f"Available methods: {[method for method in dir(users) if not method.startswith('_')]}")

Users namespace type: <class 'kili.domain_api.users.UsersNamespace'>
Available methods: ['client', 'count', 'create', 'domain_name', 'gateway', 'list', 'refresh', 'update', 'update_password']


## Test User Listing and Counting

In [ ]:
try:
    # Get current organization ID
    # Note: In a real scenario, you'd get this from your organization
    # org_id = "test-org-id"  # Replace with actual organization ID

    # Test count method
    user_count = users.count(
        # organization_id=org_id
    )
    print(f"Total users in organization: {user_count}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal in a test environment without real data")

Total users in organization: 8


In [ ]:
try:
    # Test list method - return as list
    users_list = users.list(
        # organization_id=org_id,
        first=5,
        as_generator=False,
    )
    print(f"Users (list): {users_list}")

    # Test list method - return as generator
    users_gen = users.list(
        # organization_id=org_id,
        first=5,
        as_generator=True,
    )
    print(f"Users (generator): {users_gen}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal in a test environment without real data")

Users (list): [{'email': 'test+edouard@kili-technology.com', 'id': 'user-2', 'firstname': 'Edouard', 'lastname': "d'Archimbaud"}, {'email': 'test+fx@kili-technology.com', 'id': 'user-4', 'firstname': 'FX', 'lastname': 'Leduc'}, {'email': 'test+pierre@kili-technology.com', 'id': 'user-3', 'firstname': 'Pierre', 'lastname': 'Marcenac'}, {'email': 'test+collab@kili-technology.com', 'id': 'user-8', 'firstname': 'Test', 'lastname': 'Collab'}, {'email': 'test+mlx@kili-technology.com', 'id': 'user-mlx', 'firstname': 'Test', 'lastname': 'MLX'}]
Users (generator): <generator object PaginatedGraphQLQuery.execute_query_from_paginated_call at 0x113bbd770>


/Users/baptisteolivier/work/projects/kili-python-sdk/recipes/../src/kili/presentation/client/user.py:93: UserWarning: tqdm has been forced disabled because its behavior is not compatible with the generator return type
  disable_tqdm = disable_tqdm_if_as_generator(as_generator, disable_tqdm)


## Test User Creation

In [ ]:
try:
    # Test user creation
    new_user = users.create(
        email="testuser@example.com",
        password="securepass123",
        organization_role="USER",
        firstname="Test",
        lastname="User",
    )
    print(f"Created user: {new_user}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal in a test environment - user creation requires valid organization")

Created user: {'id': 'cmg57m0xi0p3jav1a2kzj9uqt'}


## Test User Updates

In [ ]:
try:
    # Test user update
    updated_user = users.update(
        email="testuser@example.com", firstname="UpdatedName", lastname="UpdatedLastname"
    )
    print(f"Updated user: {updated_user}")

except Exception as e:
    print(f"Expected error (test environment): {e}")
    print("This is normal in a test environment")

## Test Password Security Validation

In [ ]:
# Test password validation without making actual API calls
# We'll test the validation logic directly

print("=== Testing Password Security Validation ===")

# Test cases for password validation
test_cases = [
    {
        "name": "Valid strong password",
        "params": {
            "email": "test@example.com",
            "old_password": "oldpass123",
            "new_password_1": "strongPass123!",
            "new_password_2": "strongPass123!",
        },
        "should_pass": True,
    },
    {
        "name": "Password too short",
        "params": {
            "email": "test@example.com",
            "old_password": "oldpass123",
            "new_password_1": "short",
            "new_password_2": "short",
        },
        "should_pass": False,
    },
    {
        "name": "Password confirmation mismatch",
        "params": {
            "email": "test@example.com",
            "old_password": "oldpass123",
            "new_password_1": "strongPass123!",
            "new_password_2": "differentPass123!",
        },
        "should_pass": False,
    },
    {
        "name": "Same as old password",
        "params": {
            "email": "test@example.com",
            "old_password": "samePass123",
            "new_password_1": "samePass123",
            "new_password_2": "samePass123",
        },
        "should_pass": False,
    },
    {
        "name": "Weak password (common)",
        "params": {
            "email": "test@example.com",
            "old_password": "oldpass123",
            "new_password_1": "password123",
            "new_password_2": "password123",
        },
        "should_pass": False,
    },
]

for test_case in test_cases:
    try:
        print(f"\nTesting: {test_case['name']}")
        # This will fail at the API level but should pass/fail validation first
        result = users.update_password(**test_case["params"])
        if test_case["should_pass"]:
            print("✓ Validation passed (API call expected to fail in test env)")
        else:
            print("✗ Should have failed validation but didn't")
    except ValueError as e:
        if not test_case["should_pass"]:
            print(f"✓ Validation correctly failed: {e}")
        else:
            print(f"✗ Validation failed unexpectedly: {e}")
    except Exception as e:
        if test_case["should_pass"]:
            print(f"✓ Validation passed, API error expected in test env: {e}")
        else:
            print(f"? Unexpected error: {e}")

## Test Email Validation

In [ ]:
print("=== Testing Email Validation ===")

email_test_cases = [
    ("valid@example.com", True, "Valid email"),
    ("user.name+tag@domain.co.uk", True, "Complex valid email"),
    ("invalid-email", False, "Missing @ symbol"),
    ("@domain.com", False, "Missing local part"),
    ("user@", False, "Missing domain"),
    ("", False, "Empty email"),
]

for email, should_pass, description in email_test_cases:
    try:
        print(f"\nTesting: {description} - '{email}'")
        # Test by trying to create a user (will fail at API but email should be validated first)
        result = users.create(email=email, password="testpass123", organization_role="USER")
        if should_pass:
            print("✓ Email validation passed (API error expected)")
        else:
            print("✗ Email validation should have failed")
    except ValueError as e:
        if not should_pass:
            print(f"✓ Email validation correctly failed: {e}")
        else:
            print(f"✗ Email validation failed unexpectedly: {e}")
    except Exception as e:
        if should_pass:
            print(f"✓ Email validation passed, API error expected: {e}")
        else:
            print(f"? Unexpected error: {e}")

## API Comparison: Legacy vs Domain Namespace

In [ ]:
print("=== API Comparison: Legacy vs Domain Namespace ===")
print()
print("LEGACY API (legacy=True):")
print("  kili.count_users(organization_id='org123')")
print("  kili.users(organization_id='org123', first=10)")
print("  kili.create_user(email='user@test.com', password='pass', ...)")
print("  kili.update_properties_in_user(email='user@test.com', firstname='John')")
print("  kili.update_password(email='user@test.com', old_password='old', ...)")
print()
print("NEW DOMAIN API (legacy=False):")
print("  kili.users.count(organization_id='org123')")
print("  kili.users.list(organization_id='org123', first=10)")
print("  kili.users.create(email='user@test.com', password='pass', ...)")
print("  kili.users.update(email='user@test.com', firstname='John')")
print("  kili.users.update_password(email='user@test.com', old_password='old', ...)")
print()
print("Benefits of Domain Namespace API:")
print("✓ Cleaner, more organized method names")
print("✓ Enhanced security validation for passwords")
print("✓ Better type hints and IDE support")
print("✓ More consistent parameter names")
print("✓ Comprehensive error handling")
print("✓ Method overloading for generator/list returns")

## Summary

This notebook demonstrates the Users Domain Namespace implementation:

1. **Cleaner API Surface**: Methods are logically grouped under `kili.users` (when legacy=False)
2. **Enhanced Security**: Password updates include comprehensive validation
3. **Better Error Handling**: Descriptive error messages and proper exception types
4. **Type Safety**: Full type annotations with runtime type checking
5. **Flexible Returns**: Methods support both generator and list return types

The implementation successfully provides a more intuitive and secure interface for user management operations while maintaining full backward compatibility through the existing legacy methods.